1. You have been contracted by the purina dog food company and flown out to their HQ in st. Louis Missouri!
2. You have been hired by a dog food companay to try to predict why some batches of their dosg food are spoiling 
    much quicker than intended!
3. Unfortunately this dong food company hasnt upgraded to the latest machinery meaning that the amounts of the five 
    preservative chemicals they are using can very lot, but which is the chemical that has the strongest effect?
4. The dog food company first mixes up a batch of preservative that contains 4 different preservative chemicals 
    (A,B,C,D) and then is completed with a filler chemical.
5. The food scientists belive one of the A,B,C or D preservatives is causing the problem, but need your help to 
    figure out which one.
6. Use Machine Learning with RF to find out which parameter had the most predictive power, thus finding out which 
    chemical causes the early spoiling!
7. so create a model and then find out how you can decide which chemical is the problem!


# Theire data looks like this:
    Pres_A: Percentage fo preservative A in the mix
    Pres_B: Percentage fo preservative B in the mix
    Pres_C: Percentage fo preservative C in the mix
    Pres_D: Percentage fo preservative D in the mix
    spoiled: Label indicating whether or not the dog food batch was spoiled

## our main task was to figure out which preservative chemical (A,B,C,D) was having an effect on the dog food being spoiled.

### so how can we use machine learning models to solve this?
1. Many machine learning models produce some sort of coefficient value for each feature involved, indicating their
    "importannce" or predictive power.
2. Remember back to the documentation lecture for this section of the course!
3. We mentioned that these tree method classifier had a .fetureImportances attributes available!
4. so we can create a model fit it on all the data, and then check which feature (preservative)was causing the 
    spoilage.
5. .featureImportances returns:
    SparseVector(4, {0:0.0026, 1:0.0089, 2:0.9686, 3:0.0199})
6. Corresponding to a features column:
    Row(features= DenseVector([4.0,2.0,12.0,3.0]),Spoiled = 1.0)


In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("tree_consult").getOrCreate()

In [2]:
data = spark.read.csv('dog_food.csv',inferSchema=True, header=True)

In [3]:
data.head(1)

[Row(A=4, B=2, C=12.0, D=3, Spoiled=1.0)]

In [4]:
from pyspark.ml.feature import VectorAssembler

In [5]:
data.columns

['A', 'B', 'C', 'D', 'Spoiled']

In [6]:
assembler = VectorAssembler(inputCols=['A','B','C','D'],outputCol='features')

In [7]:
output = assembler.transform(data)

In [8]:
from pyspark.ml.classification import RandomForestClassifier

In [10]:
rfc = RandomForestClassifier(labelCol='Spoiled',featuresCol='features')

In [11]:
output.printSchema()

root
 |-- A: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- C: double (nullable = true)
 |-- D: integer (nullable = true)
 |-- Spoiled: double (nullable = true)
 |-- features: vector (nullable = true)



In [12]:
final_data = output.select(['features','Spoiled'])

In [13]:
final_data.show()

+-------------------+-------+
|           features|Spoiled|
+-------------------+-------+
| [4.0,2.0,12.0,3.0]|    1.0|
| [5.0,6.0,12.0,7.0]|    1.0|
| [6.0,2.0,13.0,6.0]|    1.0|
| [4.0,2.0,12.0,1.0]|    1.0|
| [4.0,2.0,12.0,3.0]|    1.0|
|[10.0,3.0,13.0,9.0]|    1.0|
| [8.0,5.0,14.0,5.0]|    1.0|
| [5.0,8.0,12.0,8.0]|    1.0|
| [6.0,5.0,12.0,9.0]|    1.0|
| [3.0,3.0,12.0,1.0]|    1.0|
| [9.0,8.0,11.0,3.0]|    1.0|
|[1.0,10.0,12.0,3.0]|    1.0|
|[1.0,5.0,13.0,10.0]|    1.0|
|[2.0,10.0,12.0,6.0]|    1.0|
|[1.0,10.0,11.0,4.0]|    1.0|
| [5.0,3.0,12.0,2.0]|    1.0|
| [4.0,9.0,11.0,8.0]|    1.0|
| [5.0,1.0,11.0,1.0]|    1.0|
|[4.0,9.0,12.0,10.0]|    1.0|
| [5.0,8.0,10.0,9.0]|    1.0|
+-------------------+-------+
only showing top 20 rows



In [14]:
rfc_model = rfc.fit(final_data)

In [15]:
final_data.head(2)

[Row(features=DenseVector([4.0, 2.0, 12.0, 3.0]), Spoiled=1.0),
 Row(features=DenseVector([5.0, 6.0, 12.0, 7.0]), Spoiled=1.0)]

In [16]:
rfc_model.featureImportances

SparseVector(4, {0: 0.0176, 1: 0.0169, 2: 0.9464, 3: 0.0191})